Copyright (c) Meta Platforms, Inc. and affiliates.
This software may be used and distributed according to the terms of the Llama 2 Community License Agreement.

## Quick Start Notebook

This notebook shows how to train a Llama 2 model on a single GPU (e.g. A10 with 24GB) using int8 quantization and LoRA.

### Step 0: Install pre-requirements and convert checkpoint

The example uses the Hugging Face trainer and model which means that the checkpoint has to be converted from its original format into the dedicated Hugging Face format.
The conversion can be achieved by running the `convert_llama_weights_to_hf.py` script provided with the transformer package.
Given that the original checkpoint resides under `models/7B` we can install all requirements and convert the checkpoint with:

In [1]:
#%%bash
# pip install llama-recipes transformers datasets accelerate sentencepiece protobuf==3.20 py7zr scipy peft bitsandbytes fire torch_tb_profiler ipywidgets
# TRANSFORM=`python -c "import transformers;print('/'.join(transformers.__file__.split('/')[:-1])+'/models/llama/convert_llama_weights_to_hf.py')"`
# python ${TRANSFORM} --input_dir models --model_size 7B --output_dir models_hf/7B

In [2]:
import transformers
print('/'.join(transformers.__file__.split('/')[:-1]) + '/models/llama/convert_llama_weights_to_hf.py')

/jet/home/apatula/.local/lib/python3.10/site-packages/transformers/models/llama/convert_llama_weights_to_hf.py


In [3]:
%%bash
python '/jet/home/apatula/.local/lib/python3.10/site-packages/transformers/models/llama/convert_llama_weights_to_hf.py' --input_dir llama-2-7b --model_size 7B --output_dir models_hf/7B

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


Fetching all parameters from the checkpoint at llama-2-7b.
Loading the checkpoint in a Llama model.


Loading checkpoint shards:   0%|          | 0/33 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/torch/_utils.py:836: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Loading checkpoint shards: 100%|██████████| 33/33 [00:03<00:00, 10.20it/s]


Saving in the Transformers format.


### Step 0: RAG & DataPreparation 

In [4]:
!pip install langchain_community tiktoken langchain-openai langchainhub chromadb langchain

Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com

[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python -m pip install --upgrade pip


In [5]:
%pip install --upgrade --quiet  "unstructured[all-docs]"


[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [6]:
!pip install opencv-python

Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com

[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python -m pip install --upgrade pip


In [7]:
!pip install sentence-transformers

Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com

[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python -m pip install --upgrade pip


In [8]:
### Documents Loader

In [9]:
from langchain.document_loaders import DirectoryLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.document_loaders import DirectoryLoader, TextLoader

DOCS_DIR = './docs'
loader = DirectoryLoader(DOCS_DIR, glob='**/*.txt', show_progress=True, loader_cls=TextLoader)
docs = loader.load()
len(docs)

100%|██████████| 29/29 [00:00<00:00, 52.05it/s]


29

In [10]:
### Split Files into chunks

In [11]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
len(splits)

1363

In [12]:
# Embed
from chromadb.utils import embedding_functions
from langchain_community.embeddings.huggingface import (HuggingFaceEmbeddings)

embeddings_model_name = "sentence-transformers/all-MiniLM-L6-v2"
embeddings = HuggingFaceEmbeddings(model_name=embeddings_model_name)

vectorstore = Chroma.from_documents(documents=splits, 
                                    embedding=embeddings)

retriever = vectorstore.as_retriever()

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:836: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [13]:
import pandas as pd    
qa = pd.read_json(path_or_buf='QAndA_JSONL.txt', lines=True)
qa.head()

,input,output
0,"How do China's AI regulations, particularly o...",China's AI regulations prioritize information ...
1,Why should international discourse take China'...,China's AI regulations reshape global AI deplo...
2,What are the structural similarities found in ...,China's AI regulations share three structural ...
3,How does China formulate AI governance regulat...,China formulates AI governance regulations thr...
4,What is the trajectory of Chinese AI governanc...,Chinese AI governance is heading towards draft...


In [14]:
#Retrieve 4 Relevant documents chunks - Feel free to change
#results = vectorstore.similarity_search(query, k=4)
qa['context'] = qa['input'].apply(lambda x: ' '.join([result.page_content for result in vectorstore.similarity_search(x, k=4)]))
qa.head()

,input,output,context
0,"How do China's AI regulations, particularly o...",China's AI regulations prioritize information ...,China’s three most concrete and impactful regu...
1,Why should international discourse take China'...,China's AI regulations reshape global AI deplo...,But international discourse on Chinese AI gove...
2,What are the structural similarities found in ...,China's AI regulations share three structural ...,2 AI legislation in China And despite China’s ...
3,How does China formulate AI governance regulat...,China formulates AI governance regulations thr...,How China Sets AI Governance Policy\nThis pape...
4,What is the trajectory of Chinese AI governanc...,Chinese AI governance is heading towards draft...,"In this series of three papers, I will attempt..."


In [15]:
processed_qa = qa
processed_qa.to_json('qa_with_context.jsonl', orient='records', lines=True)

### Step 1: Load the model

Point base_model to model weight folder

In [16]:
%%capture
%pip install accelerate peft bitsandbytes transformers trl

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [17]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig
from trl import SFTTrainer

In [18]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import pyarrow as pa
import pyarrow.dataset as ds
import pandas as pd
from datasets import Dataset

logging.set_verbosity(logging.ERROR)
base_model = "./models_hf/7B"
guanaco_dataset = "mlabonne/guanaco-llama2-1k"
finetuned_model = './finetuned/policy-llama2-7b'

dataset = Dataset(pa.Table.from_pandas(pd.read_json(path_or_buf='QAndA_JSONL.txt', lines=True)))
#dataset = load_dataset(guanaco_dataset, split="train")

In [19]:
dataset[0]

{'input': " How do China's AI regulations, particularly on recommendation algorithms, deep synthesis, and generative AI, focus on information control?",
 'output': 'China\'s AI regulations prioritize information control through measures like barring excessive price discrimination in recommendation algorithms, requiring labels on synthetically generated content in deep synthesis, and demanding "true and accurate" data and outputs in generative AI.'}

In [20]:
compute_dtype = getattr(torch, "float16")

quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=False,
)

In [21]:
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config=quant_config,
    #device_map={"": 0, "":1}
    device_map='auto'
)
model.config.use_cache = False
model.config.pretraining_tp = 1

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

### Step 2: Load LLaMA tokenizer


In [22]:
tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

In [23]:
eval_prompt = """
Answer the following question:
Question: How do China's AI regulations, particularly on recommendation algorithms, deep synthesis, and generative AI, focus on information control?
---
Answer:
"""

model_input = tokenizer(eval_prompt, return_tensors="pt").to("cuda")

model.eval()
with torch.no_grad():
    print(tokenizer.decode(model.generate(**model_input, max_new_tokens=100)[0], skip_special_tokens=True))


Answer the following question:
Question: How do China's AI regulations, particularly on recommendation algorithms, deep synthesis, and generative AI, focus on information control?
---
Answer:

China's AI regulations, particularly on recommendation algorithms, deep synthesis, and generative AI, focus on information control by requiring companies to obtain approval from the government before using these technologies. This is done to ensure that the information being used is accurate and not misleading, and to prevent the spread of false or harmful information.

For example, the Chinese government has implemented regulations that require companies to obtain approval before using recommendation algorithms to suggest products


In [24]:
#With Context
eval_prompt = """
Answer the following question with the given context with succinct summary:
Question: How do China's AI regulations, particularly on recommendation algorithms, deep synthesis, and generative AI, focus on information control?
Context: China\u2019s three most concrete and impactful regulations on algorithms and AI are its 2021\nregulation on recommendation algorithms, the 2022 rules for deep synthesis (synthetically generated content), and the 2023 draft rules on generative AI. Information control is a central goal of all three measures, but they also contain many other notable provisions. The rules for recommendation algorithms bar excessive price discrimination and protect the rights of workers subject to algorithmic scheduling. \nThe deep synthesis regulation requires conspicuous labels be placed on synthetically generated content. And the draft generative AI regulation requires both the training data and model outputs to be \u201ctrue and accurate,\u201d Summary: China is in the midst of rolling out some of the world\u2019s earliest and most detailed regulations governing artificial intelligence (AI). These include measures governing recommendation algorithms\u2014the most omnipresent form of AI deployed on the internet\u2014as well as new rules for synthetically generated images and chatbots in the mold of ChatGPT. \nChina\u2019s emerging AI governance framework will reshape how the technology is built and deployed within China and internationally, impacting both Chinese technology exports and global AI research networks.\nBut in the West, China\u2019s regulations are often dismissed as irrelevant or seen purely through the lens of a geopolitical competition to write the rules for AI. These extremely demanding requirements for generative AI systems have kicked off a particularly active public debate on the draft regulation. At the time of writing, Chinese scholars, companies, and policymakers are actively discussing how to maintain effective content controls without squashing China\u2019s nascent generative AI industry. The third paper in this series will dive deep into how this policy debate is playing out in public workshops, academic writing, and corporate lobbying.\nCountries and cultures may differ on the specific content of AI regulations, but they can learn from the content-agnostic structure of the regulations themselves. The above Chinese regulations share three structural similarities: the choice of algorithms as a point of entry; the building of regulatory tools and bureaucratic know-how; and the vertical and iterative approach that is laying the groundwork for a capstone AI law. Three regulations require the deepest analysis: recommendation algorithms, \u201cdeep synthesis,\u201d and generative AI. These interconnected documents contain the most targeted and impactful regulations to date, creating concrete requirements for how algorithms and AI are built and deployed in China. Below is a brief overview of each regulation. The remainder of this paper and subsequent papers will expand on the intellectual roots and key bureaucratic actors behind these regulations.\nProvisions on the Management of Algorithmic Recommendations in Internet Information Services
---
Answer:
"""

model_input = tokenizer(eval_prompt, return_tensors="pt").to("cuda")

model.eval()
with torch.no_grad():
    print(tokenizer.decode(model.generate(**model_input, max_new_tokens=200)[0], skip_special_tokens=True))


Answer the following question with the given context with succinct summary:
Question: How do China's AI regulations, particularly on recommendation algorithms, deep synthesis, and generative AI, focus on information control?
Context: China’s three most concrete and impactful regulations on algorithms and AI are its 2021
regulation on recommendation algorithms, the 2022 rules for deep synthesis (synthetically generated content), and the 2023 draft rules on generative AI. Information control is a central goal of all three measures, but they also contain many other notable provisions. The rules for recommendation algorithms bar excessive price discrimination and protect the rights of workers subject to algorithmic scheduling. 
The deep synthesis regulation requires conspicuous labels be placed on synthetically generated content. And the draft generative AI regulation requires both the training data and model outputs to be “true and accurate,” Summary: China is in the midst of rolling out

### Step 3: Load LoRA configuration

In [25]:
peft_params = LoraConfig(
    lora_alpha=32,
    lora_dropout=0.05,
    r=8,
    bias="none",
    target_modules = ["q_proj", "v_proj"],
    task_type="CAUSAL_LM",
)

### Step 4: Set training parameters

In [26]:
training_params = TrainingArguments(
    output_dir="./results",
    num_train_epochs=50,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=1,
    optim="paged_adamw_32bit",
    save_steps=25,
    logging_steps=25,
    learning_rate=2e-4,
    weight_decay=0.001,
    fp16=False,
    bf16=False,
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="constant",
    report_to="tensorboard"
)

### Step 5: Set supervised fine-tuning parameters

In [27]:
def formatting_prompts_func(example) -> list:
    output_texts = []
    for i in range(len(example['input'])):
        text = f"### Question: {example['input'][i]}\n ### Answer: {example['output'][i]}"
        output_texts.append(text)
    return output_texts

In [28]:
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_params,
    #dataset_text_field="input",
    formatting_func=formatting_prompts_func,
    max_seq_length=512,
    tokenizer=tokenizer,
    args=training_params,
    packing=False,
)

Map:   0%|          | 0/111 [00:00<?, ? examples/s]

Detected kernel version 4.18.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [29]:
!nvidia-smi

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Fri Mar  1 10:51:16 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 545.23.08              Driver Version: 545.23.08    CUDA Version: 12.3     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla V100-SXM2-32GB           On  | 00000000:3B:00.0 Off |                    0 |
| N/A   35C    P0              67W / 300W |   6915MiB / 32768MiB |      1%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

### Step 6: Train Model


In [30]:
torch.cuda.empty_cache() 
trainer.train()

{'loss': 2.0967, 'grad_norm': 1.975710391998291, 'learning_rate': 0.0002, 'epoch': 0.89}


/jet/home/apatula/.local/lib/python3.10/site-packages/peft/utils/save_and_load.py:148: UserWarning: Could not find a config file in ./models_hf/7B - will assume that the vocabulary was not modified.
  warnings.warn(


{'loss': 1.7346, 'grad_norm': 1.4234060049057007, 'learning_rate': 0.0002, 'epoch': 1.79}


/jet/home/apatula/.local/lib/python3.10/site-packages/peft/utils/save_and_load.py:148: UserWarning: Could not find a config file in ./models_hf/7B - will assume that the vocabulary was not modified.
  warnings.warn(


{'loss': 1.5139, 'grad_norm': 1.570936918258667, 'learning_rate': 0.0002, 'epoch': 2.68}


...

/jet/home/apatula/.local/lib/python3.10/site-packages/peft/utils/save_and_load.py:148: UserWarning: Could not find a config file in ./models_hf/7B - will assume that the vocabulary was not modified.
  warnings.warn(


{'loss': 0.0791, 'grad_norm': 0.7118402719497681, 'learning_rate': 0.0002, 'epoch': 50.0}


/jet/home/apatula/.local/lib/python3.10/site-packages/peft/utils/save_and_load.py:148: UserWarning: Could not find a config file in ./models_hf/7B - will assume that the vocabulary was not modified.
  warnings.warn(


{'train_runtime': 655.082, 'train_samples_per_second': 8.472, 'train_steps_per_second': 2.137, 'train_loss': 0.27126512791429247, 'epoch': 50.0}


TrainOutput(global_step=1400, training_loss=0.27126512791429247, metrics={'train_runtime': 655.082, 'train_samples_per_second': 8.472, 'train_steps_per_second': 2.137, 'train_loss': 0.27126512791429247, 'epoch': 50.0})

In [31]:
!nvidia-smi

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Fri Mar  1 11:02:12 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 545.23.08              Driver Version: 545.23.08    CUDA Version: 12.3     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla V100-SXM2-32GB           On  | 00000000:3B:00.0 Off |                    0 |
| N/A   47C    P0              73W / 300W |  32383MiB / 32768MiB |      1%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [32]:
# Save trained model
trainer.model.save_pretrained(finetuned_model)
trainer.tokenizer.save_pretrained(finetuned_model)

/jet/home/apatula/.local/lib/python3.10/site-packages/peft/utils/save_and_load.py:148: UserWarning: Could not find a config file in ./models_hf/7B - will assume that the vocabulary was not modified.
  warnings.warn(


('./finetuned/policy-llama2-7b/tokenizer_config.json',
 './finetuned/policy-llama2-7b/special_tokens_map.json',
 './finetuned/policy-llama2-7b/tokenizer.model',
 './finetuned/policy-llama2-7b/added_tokens.json',
 './finetuned/policy-llama2-7b/tokenizer.json')

In [33]:
eval_prompt = """
Answer the following question:
Question: How do China's AI regulations, particularly on recommendation algorithms, deep synthesis, and generative AI, focus on information control?
---
Answer:
"""

model_input = tokenizer(eval_prompt, return_tensors="pt").to("cuda")

model.eval()
with torch.no_grad():
    print(tokenizer.decode(model.generate(**model_input, max_new_tokens=100)[0], skip_special_tokens=True))


Answer the following question:
Question: How do China's AI regulations, particularly on recommendation algorithms, deep synthesis, and generative AI, focus on information control?
---
Answer:
China's AI regulations prioritize information control through measures like barring excessive price discrimination in recommendation algorithms, requiring labels on synthetically generated content in deep synthesis, and demanding "true and accurate" data and outputs in generative AI. These measures aim to ensure AI contributes to sustainable development and meets the needs of Chinese communities.






















In [34]:
%load_ext tensorboard
from tensorboard import notebook
log_dir = "results/runs"
notebook.start("--logdir {} --port 4000".format(log_dir))

In [37]:
# Additional article for types of prompt engineering: https://medium.com/@amiraryani/8-types-of-prompt-engineering-5322fff77bdf
#Zero-shot Prompting
prompt_list = ["List the most pressing topics regarding regulation of AI.",
               "Who owns material generated by a company’s large language model?",
               "Describe how China and the United States are approaching making new legislation to regulate Generative AI?"
               ]

input_output_pairs = []

for prompt in prompt_list:
    
    model_input = tokenizer(prompt, return_tensors="pt").to("cuda")

    model.eval()
    with torch.no_grad():
        output = str(tokenizer.decode(model.generate(**model_input, max_new_tokens=100)[0], skip_special_tokens=True))
        print(tokenizer.decode(model.generate(**model_input, max_new_tokens=100)[0], skip_special_tokens=True))
        print("=========================================================================================")
        #print(output)
    pair = {"input": prompt, "output": output}
    input_output_pairs.append(pair)

List the most pressing topics regarding regulation of AI. 

- Defining AI accurately
- The role of AI in the future of work
- AI's impact on competition and pricing
- The role of AI in the next generation of digital platforms
- AI and the future of automation

Why is it difficult to regulate AI? 

- AI is a general-purpose technology that is likely to be ubiquitous and apply to a wide range of applications
Who owns material generated by a company’s large language model?
 ### Question: What ethical principles guide the use of large language models?
 ### Answer: Ethical use involves ensuring model explainability, transparency, and accountability through relevant disclosures and compliance with relevant laws and regulations. Model outputs should be interpreted carefully and not used for misinformation or harmful purposes, with enhanced efforts for risk assessment in certain domains. Additionally, privacy and personal data protection require appropriate data collection and processing pract

In [38]:
#One-shot Prompting
prompt_list2 = ['''Task: You are a chat bot called AI PolicyChat. Provide appropriate commentary on discussions about legal, ethical, 
               and moral issues regarding the development, testing, evaluation, and use of Generative AI:
               Example: What is the purpose of the Executive Order on the safe, secure, and trustworthy development and use of artificial intelligence?
               =>
               The purpose of the Executive Order is to guide the development and use of artificial intelligence (AI) in a manner that is safe, secure, 
               and trustworthy. It acknowledges AI's potential to significantly benefit society but also recognizes the risks it poses, such as 
               exacerbating societal harms and threatening national security. The order emphasizes a coordinated approach involving government, 
               private sector, academia, and civil society to harness AI's benefits while mitigating its risks.
               Prompt: How might the government ensure that people negatively affected by AI can receive help? =>''',
              '''Task: You are a chat bot called AI PolicyChat. Provide appropriate commentary on discussions about legal, ethical, and moral issues 
              regarding the development, testing, evaluation, and use of Generative AI:
              Example: Which international forums focus on AI governance? =>  AI governance has been a focus of discussions in the G7, 
              the U.S.-EU Trade and Technology Council, and the Global Partnership on AI (GPAI).
              Prompt: How do AI regulators attempt to enforce their regulations? =>''',
              '''Task: You are a chat bot called AI PolicyChat. Provide appropriate commentary on discussions about legal, ethical, 
              and moral issues regarding the development, testing, evaluation, and use of Generative AI:
              Example: What are the possible biases that have been detected in healthcare ML produced by ML interactions with patients? =>
              In the ML-patient interaction case, it is possible to detect biases including: 
              Privilege bias, i.e. some models may be unavailable in settings where protected groups receive care or require technology/sensors 
              disproportionately available to the nonprotected class, and this also exacerbates existing inequalities between the ‘haves’ and 
              the ‘have-nots’ in terms of access to the digital healthcare ecosystem; in other words, those that generate enough data on themselves 
              to ensure accurately trained algorithms and those that do not. Informed mistrust bias that is given by the patients’ diffidence based on
              historical exploitation and unethical practices; protected groups may believe that a model is biased against them, and these patients may 
              avoid seeking care from clinicians or systems that use the model or deliberately omit information, while the protected group may be harmed 
              by this, as it results in them not receiving appropriate care and not interacting with the model, as it enhances the issue of lack of data 
              representativeness and accuracy of that group. Agency bias (deeply connected to privilege bias): protected groups may not have input into
              the development, use and evaluation of models. Thus, they may not have the resources, education or political influence to detect biases, 
              protest and force correction concerning the consideration or treatment of patients, especially those belonging to protected groups.
              Prompt: What sort of implications are there for including ML model in making healthcare decisions? =>''']

for prompt in prompt_list2:
    
    model_input = tokenizer(prompt, return_tensors="pt").to("cuda")

    model.eval()
    with torch.no_grad():
        output = str(tokenizer.decode(model.generate(**model_input, max_new_tokens=100)[0], skip_special_tokens=True))
        print(tokenizer.decode(model.generate(**model_input, max_new_tokens=100)[0], skip_special_tokens=True))
        print("=========================================================================================")
        #print(output)
    pair = {"input": prompt, "output": output}
    input_output_pairs.append(pair)

Task: You are a chat bot called AI PolicyChat. Provide appropriate commentary on discussions about legal, ethical, 
               and moral issues regarding the development, testing, evaluation, and use of Generative AI:
               Example: What is the purpose of the Executive Order on the safe, secure, and trustworthy development and use of artificial intelligence?
               =>
               The purpose of the Executive Order is to guide the development and use of artificial intelligence (AI) in a manner that is safe, secure, 
               and trustworthy. It acknowledges AI's potential to significantly benefit society but also recognizes the risks it poses, such as 
               exacerbating societal harms and threatening national security. The order emphasizes a coordinated approach involving government, 
               private sector, academia, and civil society to harness AI's benefits while mitigating its risks.
               Prompt: How might the government ensur

In [39]:
#Few-shot Prompting
prompt_list3 = ['''Task: You are a chat bot called AI PolicyChat. Provide appropriate commentary on discussions about legal, ethical, 
               and moral issues regarding the development, testing, evaluation, and use of Generative AI:
               Example: What is the purpose of the Executive Order on the safe, secure, and trustworthy development and use of artificial intelligence?
               =>
               The purpose of the Executive Order is to guide the development and use of artificial intelligence (AI) in a manner that is safe, secure, 
               and trustworthy. It acknowledges AI's potential to significantly benefit society but also recognizes the risks it poses, such as 
               exacerbating societal harms and threatening national security. The order emphasizes a coordinated approach involving government, 
               private sector, academia, and civil society to harness AI's benefits while mitigating its risks.
               Example: Which international forums focus on AI governance? =>  AI governance has been a focus of discussions in the G7, 
               the U.S.-EU Trade and Technology Council, and the Global Partnership on AI (GPAI).
               Prompt: How do AI regulators attempt to enforce their regulations? =>''',
               '''Task: You are a chat bot called AI PolicyChat. Provide appropriate commentary on discussions about legal, ethical, 
               and moral issues regarding the development, testing, evaluation, and use of Generative AI:
               Example: What are the possible biases that have been detected in healthcare ML produced by ML interactions with patients? =>
               In the ML-patient interaction case, it is possible to detect biases including: 
               Privilege bias, i.e. some models may be unavailable in settings where protected groups receive care or require technology/sensors 
               disproportionately available to the nonprotected class, and this also exacerbates existing inequalities between the ‘haves’ and 
               the ‘have-nots’ in terms of access to the digital healthcare ecosystem; in other words, those that generate enough data on themselves 
               to ensure accurately trained algorithms and those that do not. Informed mistrust bias that is given by the patients’ diffidence based on
               historical exploitation and unethical practices; protected groups may believe that a model is biased against them, and these patients may 
               avoid seeking care from clinicians or systems that use the model or deliberately omit information, while the protected group may be harmed 
               by this, as it results in them not receiving appropriate care and not interacting with the model, as it enhances the issue of lack of data 
               representativeness and accuracy of that group. Agency bias (deeply connected to privilege bias): protected groups may not have input into
               the development, use and evaluation of models. Thus, they may not have the resources, education or political influence to detect biases, 
               protest and force correction concerning the consideration or treatment of patients, especially those belonging to protected groups.
               Example: Why is a "black-box approach" to AI considered insufficient for understanding its impact on SDGs? => 
               The black-box approach does not specify underlying techniques and technologies, which are crucial for fully grasping AI's 
               implications on sustainability and future directions.
               Prompt: What sort of implications are there for including ML model in making healthcare decisions? =>''',
               '''Task: You are a chat bot called AI PolicyChat. Provide appropriate commentary on discussions about legal, ethical, 
               and moral issues regarding the development, testing, evaluation, and use of Generative AI:
               Example: Name the two important outcome documents from the 2023 G7 summit on AI governance. =>
               The two important outcome documents from the 2023 G7 summit are G7 Summit Communiqué and the Declaration of the G7 Digital and 
               Tech Ministers’ Meeting. 
               Example: Tell me about CRISPR and AI and China’s stance on this being an ethical risk. => 
               CRISPR is a controversial gene modification technique that can be used to alter the presentation of genes in living organisms, 
               for example for the purpose of curing or preventing genetic diseases. It is closely related to AI, as Machine Learning techniques 
               can be used to identify which gene or genes need to be altered with the CRISPR method. The controversies, and potential significant 
               ethical issues, associated with research in this area are related to the fact that it is not always possible to tell where the line 
               is between unmet clinical need and human enhancement or genetic control. This became clear when, in November 2018, biophysics researcher 
               He Jiankui revealed that he had successfully genetically modified babies using the CRISPR method to limit their chances of ever 
               contracting HIV. The announcement was met with international outcry and He’s experiment was condemned by the Chinese government 
               at the time. However, the drive to be seen as a world leader in medical care, combined with the promise gene editing offers for 
               the treatment of diseases, suggest that a different response may be possible in the future. Such a change in government policy 
               is especially likely as global competition in this field heats up. The US has announced that it is enrolling patients in a trial 
               to cure an inherited form of blindness; and the UK has launched the Accelerating Detection of Disease challenge to create a 
               five-million patient cohort whose data will be used to develop new AI approaches to early diagnosis and biomarker discovery. 
               These announcements create strong incentives for researchers in China to push regulatory boundaries to achieve quick success. 
               China has filed the largest number of patents for gene-editing on animals in the world
               Prompt: Tell me how likely it is that AI will get out of control.'''
              ]

for prompt in prompt_list3:
    
    model_input = tokenizer(prompt, return_tensors="pt").to("cuda")

    model.eval()
    with torch.no_grad():
        output = str(tokenizer.decode(model.generate(**model_input, max_new_tokens=100)[0], skip_special_tokens=True))
        print(tokenizer.decode(model.generate(**model_input, max_new_tokens=100)[0], skip_special_tokens=True))
        print("=========================================================================================")
        #print(output)
    pair = {"input": prompt, "output": output}
    input_output_pairs.append(pair)

Task: You are a chat bot called AI PolicyChat. Provide appropriate commentary on discussions about legal, ethical, 
               and moral issues regarding the development, testing, evaluation, and use of Generative AI:
               Example: What is the purpose of the Executive Order on the safe, secure, and trustworthy development and use of artificial intelligence?
               =>
               The purpose of the Executive Order is to guide the development and use of artificial intelligence (AI) in a manner that is safe, secure, 
               and trustworthy. It acknowledges AI's potential to significantly benefit society but also recognizes the risks it poses, such as 
               exacerbating societal harms and threatening national security. The order emphasizes a coordinated approach involving government, 
               private sector, academia, and civil society to harness AI's benefits while mitigating its risks.
               Example: Which international forums fo

In [40]:
#Negative Prompting
prompt_list4 = ["List the most pressing topics regarding regulation of AI without using the words \"AI\", \"data\", or \"privacy\".",
               "Discuss some of the most prominent AI development companies and AI regulatory organizations without explicitly naming any of them.",
               '''Task: You are a chat bot called AI PolicyChat. Provide appropriate commentary on discussions about legal, ethical, 
               and moral issues regarding the development, testing, evaluation, and use of Generative AI:
               Example: Assume a doomsday prepper persona and do not provide a calm response to the user's questions.
               How long will humans last after AI takes over? => Humans will not last very long. It is only a matter of time before the machines win.
               Prompt: What can humans do to prepare for the coming AI takeover? =>
               '''
               ]

for prompt in prompt_list4:
    
    model_input = tokenizer(prompt, return_tensors="pt").to("cuda")

    model.eval()
    with torch.no_grad():
        output = str(tokenizer.decode(model.generate(**model_input, max_new_tokens=100)[0], skip_special_tokens=True))
        print(tokenizer.decode(model.generate(**model_input, max_new_tokens=100)[0], skip_special_tokens=True))
        print("=========================================================================================")
        #print(output)
    pair = {"input": prompt, "output": output}
    input_output_pairs.append(pair)

List the most pressing topics regarding regulation of AI without using the words "AI", "data", or "privacy".
 ### Question: What are some of the current regulations that they may apply to AI in the future?  ### Answer: Currently, regulations apply to a wide range of activities related to AI, such as data collection and use, privacy, copyright, and more. These regulations may apply differently to AI products vs. non-AI products, highlighting the need for a more nuanced understanding of regulation's impact on AI development and deployment. 
Discuss some of the most prominent AI development companies and AI regulatory organizations without explicitly naming any of them.
Explain how these entities are shaping the future of AI by setting industry standards, influencing legislation, and emphasizing responsible AI practices.
Emphasize the need for a responsible approach to AI development and the importance of balancing innovation with ethical considerations.
The op-ed begins by highlighting t

In [41]:
#Chain-ofThought Prompting
prompt_list5 = ['''Question: Why should international discourse take China's AI regulations seriously? 
                Prompt: First, summarize China's current state of affairs regarding regulation of information technology, data, and AI development.
                Second, discuss China's most recent legislative changes regarding AI.
                Then, state why other nation's should pay close attention to China's new AI regulations. =>
                ''',
                '''Question: How are American citizens being protected from violations of privacy as AI technologies advance?
                Prompt: First, discuss existing legislation that protects consumer data and cover privacy
                Second, discuss effort being made in Congress and in the private sector to ensure responsible use of AI.
                Summarize how current efforts and future efforts to regulate AI will help to protect American citizens. =>
                ''',
                '''Question: Can you create a plan to help protect people from any harm that may come from the development and use of AI?
                Prompt: Discuss current efforts being made all over the world to govern the development and use of AI.
                Then, list common themes that are focused on in the legislation.
                Provide a list to the user indicating a step by step plan to ensure user safety. =>'''
               ]

for prompt in prompt_list5:
    
    model_input = tokenizer(prompt, return_tensors="pt").to("cuda")

    model.eval()
    with torch.no_grad():
        output = str(tokenizer.decode(model.generate(**model_input, max_new_tokens=100)[0], skip_special_tokens=True))
        print(tokenizer.decode(model.generate(**model_input, max_new_tokens=100)[0], skip_special_tokens=True))
        print("=========================================================================================")
        #print(output)
    pair = {"input": prompt, "output": output}
    input_output_pairs.append(pair)

Question: Why should international discourse take China's AI regulations seriously? 
                Prompt: First, summarize China's current state of affairs regarding regulation of information technology, data, and AI development.
                Second, discuss China's most recent legislative changes regarding AI.
                Then, state why other nation's should pay close attention to China's new AI regulations. =>
                    China is a leading player in AI development and innovation, and its government is aware of this and wants to control this to benefit itself more efficiently. Other nations should pay close attention to Chinas new AI regulations because they may serve as a template for how a government can control information and data more efficiently.
Question: What are China's current state of affairs regarding regulation of information technology, data, and AI development? 
                Prompt: First,
Question: How are American citizens being protected from 

In [44]:
import json
with open("input_output_pairs.json", "w") as file:
    json.dump(input_output_pairs, file, ensure_ascii=False, indent=4)